# generate_statistics

In [137]:
import mlrun

## 1. Configuration

In [138]:
project_name = "debug"
image = "registry-service.mlrun.svc.cluster.local/mlrun/mlrun:1.9.1"

dataset_file = "data/sample-dataset.parquet"
tfdv_schema_file = "assets/tfdv_schema_training.pbtxt"

In [139]:
# get the debug project
project = mlrun.get_or_create_project(
    name=project_name,
    context="./",
    user_project=True
)

> 2025-08-22 16:06:50,893 [info] Project loaded successfully: {"project_name":"debug-johannes"}


### 1.1 Project Prerequisites

### 1.1.1 Schema

In [140]:
# load the schema artefact
project.log_artifact(
    item="train-schema",
    local_path=tfdv_schema_file,
    format="pbtxt"
)

In [141]:
schema_artifact = project.get_artifact("train-schema")

#### 1.1.2 Dataset 

In [142]:
import pandas as pd

# load the dataset
df = pd.read_parquet(dataset_file)

# log the dataset
dataset_artifact = project.log_dataset(
    key="train-data",
    df=df
)

## 2. Run Locally

In [143]:
function = mlrun.code_to_function(
    name="generate_statistics",
    filename="generate_statistics.py",
    kind="job",
    image=image,
    handler="generate_statistics",
    requirements_file="requirements.txt"
)

In [144]:
result = function.run(
    inputs={
        "dataset_uri": dataset_artifact.uri,
        "schema_file": schema_artifact.uri,
    },
    params={
        "stats_options": {
            "infer_type_from_schema": True
        }
    },
    local=True,
    auto_build=True
)

> 2025-08-22 16:06:51,719 [warning] It is recommended to use k8s secret (specify secret_name), specifying aws_access_key/aws_secret_key directly is unsafe.
> 2025-08-22 16:06:51,725 [info] Storing function: {"db":"http://dragon.local:30070","name":"generate-statistics-generate-statistics","uid":"59008e593a73466996c35572757659a4"}
> 2025-08-22 16:06:51,837 [info] downloading s3://mlrun/projects/debug-johannes/artifacts/train-data.parquet to local temp file
> 2025-08-22 16:06:51,881 [info] downloading s3://mlrun/projects/debug-johannes/artifacts/train-schema.pbtxt to local temp file
> 2025-08-22 16:06:51,884 [info] Loading Dataset
> 2025-08-22 16:06:52,088 [info] Computing statistics
> 2025-08-22 16:06:53,275 [info] Generating statistics HTML


> 2025-08-22 16:06:53,977 [info] Run execution finished: {"name":"generate-statistics-generate-statistics","status":"completed"}
